In [3]:
import tensorflow as tf

In [4]:
# model = tf.keras.applications.NASNetLarge(
#     include_top=False,
#     weights="imagenet",
#     input_tensor=None,
#     input_shape=None,
#     pooling=None,
#     classes=1000,
# )
# model

In [5]:
# -*- coding: utf-8 -*-
""" 
main.py

python main.py --backbone 'efficientnetb7' --save False --desc '339_cls_TEST'

downloaded models :: 
'resnet50'
'resnet50v2'
'resnet152v2'
'efficientnetb7'
'vgg16'
"""
import argparse
import sys

sys.path.append('/workspace/detect-me/product_classifier/')
sys.path.append('/workspace/detect-me/product_classifier/utils')
from configs.config import (CFG_RESNET50,
                           CFG_RESNET50V2,
                           CFG_RESNET152,
                           CFG_VGG16,
                           CFG_EfficientNet )

from scripts.models.classifier import Classifier
import os
import tensorflow as tf
import datetime
import numpy as np
import pandas as pd  

csv_path = '/workspace/detect-me/product_classifier/sold60_12JUN.csv'

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 


%load_ext tensorboard
def parse_args(args):
    """ Parse the arguments.
    """
    parser     = argparse.ArgumentParser(description='Simple training script for training a RetinaNet network.')
    parser.add_argument('--backbone',     help='Backbone model used', default='resnet50', type=str)
    parser.add_argument('--save',         help='save.', default=False, type=bool)
    parser.add_argument('--desc',         help='description to save model and tensorboad logs .', default='untitled', type=str)

    return parser.parse_args(args)


def get_traindata_info(model):
    train_labels = model.dataset['train'].labels
    counts = np.unique(train_labels, return_counts=True)
    return pd.DataFrame(counts).T

__init__ in root product_classifier


In [6]:
backbone =      'resnet50'
should_I_save = False
desc =          'N'

if backbone=='resnet50':
    train_config = CFG_RESNET50
    base_model = tf.keras.applications.ResNet50(
                         include_top=False,
                         weights="imagenet",
                         input_tensor=None,
                         # input_shape=(*self.image_size, 3),
                         pooling=None,
                         classes=1000)
elif backbone=='resnet50v2':
    train_config = CFG_RESNET50V2
    base_model = tf.keras.applications.ResNet50V2(
                        include_top=False, weights='imagenet', input_tensor=None,
                        input_shape=None, pooling=None, classes=1000,
                        classifier_activation='softmax')
elif backbone=='resnet152v2':
    train_config=CFG_RESNET152
    base_model = tf.keras.applications.ResNet152V2(
                    include_top=False,
                    weights="imagenet",
                    input_tensor=None,
                    input_shape=None,
                    pooling=None,
                    classes=1000,
                    classifier_activation="softmax",)
elif backbone=='efficientnetb7':
    train_config = CFG_EfficientNet
    base_model = efficientNetB7 = tf.keras.applications.EfficientNetB7(
                include_top=False,
                weights="imagenet",
                input_tensor=None,
                input_shape=None,
                pooling=None,
                classes=1000,
                classifier_activation="softmax",)
elif backbone=='vgg16':
    train_config = CFG_VGG16
    base_model = tf.keras.applications.VGG16(include_top=False,
                              weights='imagenet',
                              classes=1000)
else:
    raise ValueError('saved models in ./keras folder :: efficientnetb7-vgg16-resnet152v2-resnet50-resnet50v2 ')

freeze_up_to = train_config['train']['fine_tune_at']
lr = train_config['train']['opt_lr']
batchs = train_config['train']['batch_size']
optimizer = train_config['train']['optimizer']['type']
optimizer__callback = train_config['train']['optimizer']['lr_callback']['type']

# optimizer__callback_params = train_config['train']['optimizer']['lr_callback']['params']
# optimizer__callback_params = '___'.join([f'{k}-{v}' for k,v in optimizer__callback_params.items()])

epochs = train_config['train']['epochs']
dropout = train_config['train']['dropout']
base_model_name = train_config['model']['name']
data_augmentation = '___'.join([f'{k}-{v}' for k,v in train_config['train']['data_augmentation'].items()])


time = datetime.datetime.utcnow() + datetime.timedelta(hours=4, minutes=30)
timest = time.strftime("%Y-%b-%d----%H%M%S")

desc = f"{desc}__{timest}"

model_nametag = f'{base_model_name}---(Freeze-{freeze_up_to})-(lr-{lr}-{optimizer})-(opt_cb-{optimizer__callback})-(epoch-{epochs})-(batch-{batchs})-(dropout-{dropout})-{data_augmentation}-{desc}'


In [7]:
model = Classifier(cat = 'milk',
                 config =train_config,
                 base_model = base_model,
                 name_tensorboard=model_nametag,)
            

model_path ::
/workspace/detect-me/product_classifier/saved_models/milk/resnet50
tensorboard_logs_path_ ::
/workspace/detect-me/product_classifier/saved_models/milk/tensorboard_logs
data_path_ ::
/workspace/detect-me/product_classifier/data/milk
init_done


In [9]:
%tensorboard --logdir '/workspace/detect-me/product_classifier/saved_models/milk/tensorboard_logs' --port 6666

In [10]:
print('len(base_model.layers) :\n',len(model.base_model.layers))

model.load_data(csv_path = None)
train_info = get_traindata_info(model)
model.build()
model.compile(fine_tune_at=model.fine_tune_at,
                        lr=model.lr)


history = model.train(save=should_I_save,
                      save_name=model_nametag,
                      train_info=train_info)

len(base_model.layers) :
 175
len classes_set 339
load_with OUT_mixup_generator
Found 10113 validated image filenames belonging to 339 classes.
Found 2632 validated image filenames belonging to 339 classes.
Found 718 validated image filenames belonging to 339 classes.
________________________________________________WARNING!___________________________________________
something is WRONGGGG!! num  classes in CONFIG is DIFF from what I get from Train-validation folder
num classes is calculated based on  folder structure in xxx/train-val
num_classes : 339
________________________________________________WARNING!___________________________________________
resnet50 preprocess
len(self.model.trainable_variables)  66
adam
['*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*

Process Keras_worker_ForkPoolWorker-1:
Process Keras_worker_ForkPoolWorker-2:
Process Keras_worker_ForkPoolWorker-3:
Process Keras_worker_ForkPoolWorker-4:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.8/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/usr/lib/python3.8/multiprocessing/pool.py", line 131, in worker
    put((job, i, result))
  File "/usr/lib/python3.8/multiprocessing/queues.py", line 356, in get
    res = self._reader.recv_bytes()
  File "/usr/lib/python3.8/multiprocessing/queues

KeyboardInterrupt: 

Process Keras_worker_ForkPoolWorker-5:
Process Keras_worker_ForkPoolWorker-7:
Process Keras_worker_ForkPoolWorker-8:
Process Keras_worker_ForkPoolWorker-6:
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.8/multiprocessing/pool.py", line 114, in worker
    task = ge

In [14]:
#TODO save result 
print('________________________________________________________________________')
print('____________________eval on test set____________________________________')
testt = model.evaluate(model.dataset['test'],)
print('________________________________________________________________________')

________________________________________________________________________
____________________eval on test set____________________________________
718/718 [==============================] - 10s 13ms/step - loss: 0.2931 - accuracy: 0.9359 - Precision-tr-0.5: 0.9501 - Precision-tr-0.9: 0.9765 - fp: 35.0000 - fn: 51.0000 - recall-tr--0.5: 0.9290 - recall-tr--0.9: 0.8691
________________________________________________________________________


'/workspace/detect-me/product_classifier/saved_models/'

In [24]:
model.base_model.name

'resnet50'

In [31]:
import pathlib

UsageError: Missing filename.


In [40]:
cat='milk'

In [41]:
if True:
    path_to_save_model = pathlib.Path((train_config['data']['path_to_model']))/ cat/ model.base_model.name /name_tag   # save model history

with open(f'{path_to_save_model}/test.json', 'w') as writer:
    json.dump(testt,writer,indent=4)

NameError: name 'name_tag' is not defined